In [2]:
# Import necessary packages 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
% matplotlib inline


Read the takehome user engagement data and checking for any null or missing values

In [3]:
User_engagemnt = pd.read_csv("takehome_user_engagement.csv")
print(User_engagemnt.head())

            time_stamp  user_id  visited
0  2014-04-22 03:53:30      1.0      1.0
1  2013-11-15 03:45:04      2.0      1.0
2  2013-11-29 03:45:04      2.0      1.0
3  2013-12-09 03:45:04      2.0      1.0
4  2013-12-25 03:45:04      2.0      1.0


In [4]:
User_engagemnt.isnull().sum()

time_stamp    0
user_id       1
visited       1
dtype: int64

By observing there are no missing values in user engagement data

In [5]:
# Remove deprecation warnings
import warnings
warnings.filterwarnings('ignore')
#encoding option is used to avoid uft encoding error
Users = pd.read_csv("takehome_users.csv",encoding='ISO-8859-1')

In [6]:
Users.describe()

,object_id,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
count,12000.00000,8.823000e+03,12000.000000,12000.000000,12000.000000,6417.000000
mean,6000.50000,1.379279e+09,0.249500,0.149333,141.884583,5962.957145
std,3464.24595,1.953116e+07,0.432742,0.356432,124.056723,3383.761968
min,1.00000,1.338452e+09,0.000000,0.000000,0.000000,3.000000
25%,3000.75000,1.363195e+09,0.000000,0.000000,29.000000,3058.000000
50%,6000.50000,1.382888e+09,0.000000,0.000000,108.000000,5954.000000
75%,9000.25000,1.398443e+09,0.000000,0.000000,238.250000,8817.000000
max,12000.00000,1.402067e+09,1.000000,1.000000,416.000000,11999.000000


In [7]:
Users.head()
Users.drop(columns=['last_session_creation_time'],inplace = True)
Users.count()

object_id                     12000
creation_time                 12000
name                          12000
email                         12000
creation_source               12000
opted_in_to_mailing_list      12000
enabled_for_marketing_drip    12000
org_id                        12000
invited_by_user_id             6417
dtype: int64

By observation Last_session_creation_time is not in the datatime format and lot many values are missing.

In [8]:
print('There are {} unique users of the 207,917 entries in engagement data'.format(len(Users['email'].unique())))

There are 11980 unique users of the 207,917 entries in engagement data


In this analysis, I have analyzed 12,000 users and out of the total users, 1,656 users became adopted users, which is 13.8% of the total users. The rest, non-adopted users, can be separated into two groups, never visited and visited but not adopted, which are 26.5% and 59.7% respectively. For never visited users, we should attract them to visit for the first time and for visited but not adopted users, we should improve their experience to increase adoption.



Since the prediction should be made at the time when the account is created, any factors that are acquired after the users started to using it should be avoided. Otherwise, the factors would leak information to adoption prediction. By analyzing and preprocessing the data, I have feature engineered 5 new variables to predict adoption, which are email_domain (The domain from the email address), adopted_refer (whether the user is referred by adopted user), same_org (whether the user and the person referred are in the same organization), org_size (the size of the organization) and org_adopt_pct (percentage of the people in the organization are adopted users).

As we can see from the graph above, this data set is highly imbalanced, which means we have much more non-adopted users than adopted users. If the imbalanced data is directly used, the model would tend to predict non-adopted users to achieved higher accuracy but lose the ability to identify potential adopted users. Therefore, I choose the under sampling method to combat the imbalance issue and leverage the state of art machine learning technology (xgboost) to build the prediction model. The evaluation metric is set to be AUC rather than accuracy due to the goal and imbalanced data set.

From the analysis, most of the adopted users are from a relatively small organization with high adoption rate in the organization and referred by adopted user; meanwhile, users from certain email domains have lower adoption rate than others. It could be caused by the email domain is blocking asana ads or represents certain demographics. Overall, the adoption rate in the organization plays the most important role in determining whether the new users will adopt in the future.

Due to the limitation of this analysis, I did not dive into the the factors trigger initial visit and the factors keep users engaged. What is more, the adoption prediction is made when the users create the account; however, the adoption rate changes when more factors become available such as days until first visit, visit times, days used, etc. Further analysis is highly recommended.

In [9]:
Users1 = Users['email'].str.split('@', expand=True)
Users1.head()
Users1.rename(columns={1:"company_site"},inplace=True)
